## Used Classes

1. Math
2. Random

In [1]:
import math
import random as rn

### Class **Chromosome** to store

- **binList**: Random **Binary List**
- **eq**: **Equation** to solve
- **X**: Value of **X** of *binary list*
- **Y**: Value of **Y** calculated from *Equation*

In [2]:
class Chromosome:
    def __init__(self, l: list[str], eq: str):
        self.binList: list[str] = l
        self.eq = eq
        self.X: int = int(int(''.join(self.binList), base=2))
        self.Y: float = float(eval(eq.replace("x", str(self.X))))

    def mutate(self, idx: int) -> None:
        '''
            Mutate this chromosome at a particular index
            if this chromosome is selected in mutation
            process
        '''
        self.binList[idx] = '0' if self.binList[idx] == '1' else '1'
        self.X = int(int(''.join(self.binList), base=2))
        self.Y: float = float(eval(self.eq.replace("x", str(self.X))))

Class **GeneticAlgorithm** to process equations and find maxima

1. **equation**: stores the **equation** to *process*
2. **xs**: stores all the **chromosomes**
3. **population**: stores current value of **population**
4. **genes**: stores the number of **genes** per **chromosome**
5. **percentage**: *ratio* of people undergoing **crossover**

In [3]:
class GeneticAlgorithm:
    def __init__(self, equation: str) -> None:
        self.equation: str = equation
        self.xs: list[Chromosome] = []
        self.population: int = 50
        self.genes: int = 10
        self.percentage: float = 0.8

    def _initialization(self) -> None:
        '''
            Initialisaton of chromosomes
        '''
        for i in range(self.population):
            l: list[int] = []
            for j in range(self.genes):
                num: int = rn.uniform(0, 1) - 0.5
                l.append('0' if num < 0 else '1')
            self.xs.append(Chromosome(l, self.equation))

    def _tournament_selection(self) -> None:
        '''
            Tournament Selection process
            to select the better 
            chromosomes by doing a 
            tournament
        '''
        winners: set = set()
        i: int = int(rn.random() * self.population)
        Yi: float = self.xs[i].Y

        j: int = int(rn.random() * self.population)
        Yj: float = self.xs[j].Y

        win: int = max(Yi, Yj)
        winner: int = i if Yi == win else j
        winners.add(winner)
        loser: int = abs(win - winner)

        while (len(winners) < len(self.xs) / 2):
            i: int = int(rn.random() * self.population)
            Yi: float = self.xs[i].Y

            j: int = int(rn.random() * self.population)
            Yj: float = self.xs[j].Y

            if (Yi in winners or Yj in winners):
                continue

            win = max(Yi, Yj)
            winner = i if Yi == win else j
            winners.add(winner)

        # Sorting according to the value of Y given by each chromosome
        self.xs.sort(key=lambda x: x.Y, reverse=True)

    def _crossover(self) -> None:
        '''
            Crossover Process where 2 consecutive
            parents undergo crossover to produce
            2 children
        '''
        par1_idx = 0
        par2_idx = 1
        while (par2_idx < int(self.population * self.percentage)):
            par1 = self.xs[par1_idx].binList
            par2 = self.xs[par2_idx].binList
            par1_idx += 2
            par2_idx += 2
            num = int(rn.random() * self.genes)

            # Child 1
            l: list[int] = []
            l = par1[0: num] + par2[num: ]
            self.xs.append(Chromosome(l, self.equation))

            # Child 2
            l = []
            l = par2[0: num] + par1[num: ]
            self.xs.append(Chromosome(l, self.equation))

        self.population += self.population * self.percentage

    def _mutation(self) -> None:
        '''
            Mutation process where the changes
            occurs in some chromosomes
            while changing generations
        '''
        for i in range(int(self.population * 0.1)):
            num = int(rn.random() * self.population)
            self.xs[num].mutate(int(rn.random() * self.genes))

    def _tolerance(self, lastGen: Chromosome) -> bool:
        '''
            Check the tolerence value and compare the
            best performing chromosome of two 
            consecutive generations
        '''
        return False if self.xs[0].Y == lastGen.Y else True

    def solve(self) -> int:
        '''
            Combining all the process in an order to
            perform Genetic Algorithm
        '''
        self._initialization()
        lastGen: Chromosome = self.xs[0]
        flag: bool = True
        while (flag):
            self._tournament_selection()
            self._crossover()
            self._mutation()
            self.xs.sort(key=lambda x: x.Y, reverse=True)
            self.xs = self.xs[0: 50]
            self.population = 50
            flag = self._tolerance(lastGen)
            lastGen = self.xs[0]

        return self.xs[0].Y


## Main Function

In [4]:
def main() -> None:
    eq: str = "math.sin(x)"
    ga: GeneticAlgorithm = GeneticAlgorithm(eq)
    print("Maxima = ", ga.solve())

In [5]:
if __name__ == "__main__":
    main()

Maxima =  0.9997545059076306
